# 以全连接的方式实现ＭＮＩＳＴ识别

## 加载库文件

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 定义全局变量

In [2]:
INPUT_NODE = 784 # 输入层节点数
LAYER1_NODE = 500 # 隐含层节点数
OUTPUT_NODE = 10 # 输出层节点数
BATCH_SIZE = 100 # 一个Ｂａｔｃｈ的大小：数字越小，训练过程越接近随机梯度下降；数字越大，训练过程越接近梯度下降
LEARNING_RATE_BASE = 0.8 # 基础学习率
LEARNING_RATE_DECAY = 0.9 #　学习率的衰减率
REGULARIZATION_RATE = 0.0001 # 正则化系数
TRAINING_STEPS = 30000 # 训练轮数
MOVING_AVERAGE_DECAY = 0.99 # 滑动平均衰减率

global_step = tf.Variable(0, trainable=False) # 定义存储训练轮数的变量，属性为不可训练类型

## 前向传播函数

给定神经网络的输入：输入层数据, 是否使用滑动平均模型的标志, 输入层的权重, 输入层的偏置量, 隐含层的权重, 隐含层的偏置量。在这里定义了一个使用 $ReLU$ 激活函数的三层全连接神经网络。返回:前向传播计算的结果

In [3]:
def inference(input_tensor, avg_class,  weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        # 计算隐含层前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        # 计算输出层的前向传播结果.
        # 因为在计算损失函数时，会一并计算softmax函数，此处不需要加入激活函数
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        # 首先, 使用avg_class.average函数来计算出变量的滑动平均值
        # 然后，再计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

## 训练模型的过程

In [4]:
def train(mnist):
    # 建立输入数据和标签的占位符
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    
    # 生成隐含层和输出层参数
    weight1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    bias1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    weight2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    bias2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算在当前条件下神经网路的前向传播结果
    # 此处不使用滑动平均类
    y = inference(input_tensor=x, avg_class=None, weights1=weight1, biases1=bias1, weights2=weight2, biases2=bias2)
    
    
    # 定义滑动平均类和滑动平均操作
    # 对所有代表神经网络的参数的变量上使用滑动平均类。tf.trainable_variables返回的就是图上集合ＧraphKeys.TRAINABLE_VARABLEＳ中的元素
    # 这个集合的元素就是所有没有指定trainable=False
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 重新计算前向传播结果
    # 此处使用滑动平均类：滑动平均操作不会改变变量本身的取值，而是会维护一个影子变量来记录其滑动平均值
    average_y = inference(input_tensor=x, avg_class=variable_averages, weights1=weight1, biases1=bias1, weights2=weight2, biases2=bias2)
    
    # 计算损失函数：交叉熵作为刻画预测值与损失值之间差距的损失函数
    # 这里使用TensorFlow中提供的sparse_softmax_cross_entroy_with_logitis函数来计算交叉熵。
    # 当分类问题只有一个正确答案时，可以使用这个函数来加速交叉熵的计算。这个函数的第一个参数是神经网络
    # 不包括softmax层的前向传播结果，第二个参数是训练数据的第二个答案。
    # 因为标准答案是一个长度为１０的一维数组,而该答案需要提供一个正确答案的数字，所以需要使用tf.argmax
    # 函数来得到正确答案对应的类别编号。
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_, 1), logits=y)
    
    # 计算在当前batch下，所有样例交叉熵的平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 建立Ｌ２正则化类，并计算正则化值
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weight1) + regularizer(weight2)
    
    # 计算总损失函数
    loss = cross_entropy_mean + regularization
    
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减
        global_step, # 当前迭代的轮数
        mnist.train.num_examples / BATCH_SIZE, # 过完所有的训练数据需要迭代的次数
        LEARNING_RATE_DECAY) # 学习速率的衰减速度
    
    # 优化损失函数
    # 这里的损失函数包含了交叉熵和正则化项
    train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    
    # 在训练神经网络模型时，每过一遍数据即需要通过反向传播来更新神经网络中的参数。
    # 又需要更新每一个参数的滑动平均值
    # 为了一次完成多个操作，ＴensorFlow提供了tf.control_dependencies和tf.group两种机制
    # 下面两行程序和train_op = tf.group(train_step, variables_averages_op)等价
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name = "train")
        
    # 检验使用滑动平均模型的神经网络前向传播结果是否正确
    # tf.argmax(average_y, 1):计算每个样例的预测答案
    # 其中average_y是一个batch_size*10的二维数组，每一行表示一个样例的前向传播结果。
    # tf.argmax的第二个参数"1"表示选取最大值的操作仅在第一个维度中进行，也就是说，只在没一行选取最大值对应的下标
    # 于是得到的结果时一个长度为batch的一维数组，其中的值表示识别的结果
    # tf.equal判断两个张量的每一维度是否相等，相等的元素返回Tｒue,不相等的元素返回False
    # 将一维bool型的数值转化为实数，然后计算平均值
    # 这个平均值就是模型在这一组数据上的精度
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        # 准备验证数据
        # 一般在神经网络的训练过程中会通过验证数据大致判断停止的条件和评判训练的结果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        
        # 准备测试数据
        # 在真实的应用中，这部分数据是不可见的，这个数据只是作为模型优劣最后的评判标准
        test_feed = {x: mnist.test.images, y_:mnist.test.labels}
        
        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            # 没１０００次输出一次在验证集上的结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证集上的结果
                # 因为MNIST的验证集比较小，所以一次可以处理所有的验证集。
                # 为了计算方便，本样例程序没有将验证集划分为更小的batch；
                # 当神经网络模型比较复杂或者验证数据集比较大时，太大的batch可能会导致计算时间过长或者内存溢出
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy " "using average model is %g" % (i, validate_acc))
                
            # 产生这一轮使用batch的数据集, 并训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_:ys})
            
        # 在结束训练之后，在测试数据上最终检测模型的正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print ("After %d training step(s), test accuracy " "model is %g" % (TRAINING_STEPS, test_acc))

## 主程序入口

In [5]:
def main(argv=None):
    # 下载或读取数据
    mnist = input_data.read_data_sets("../datasets/MNIST_data/", one_hot=True)
    train(mnist)
    
# TensorFloｗ提供的一个主程序入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()

Extracting ../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1352
After 1000 training step(s), validation accuracy using average model is 0.9758
After 2000 training step(s), validation accuracy using average model is 0.9804
After 3000 training step(s), validation accuracy using average model is 0.9804
After 4000 training step(s), validation accuracy using average model is 0.9814
After 5000 training step(s), validation accuracy using average model is 0.9806
After 6000 training step(s), validation accuracy using average model is 0.9804
After 7000 training step(s), validation accuracy using average model is 0.9824
After 8000 training step(s), validation accuracy using average model is 0.9814
After 9000 training step(s), validation accuracy u

SystemExit: 

/home/zach/anaconda2/envs/tensorflowpy27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
